In [1]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime
from plotly.subplots import make_subplots
pd.options.plotting.backend = "plotly"

In [161]:
zs_df, intervals_per_hour = zs_data("2022-1-1 00:00", "2022-12-31 23:45",
    src = "data/evoenergy-zone-substation-report-2021-23.csv")

pop_SA3_df = pd.read_csv('data/SA3_population.csv', index_col=0)
pop_SA2_df = pd.read_csv('data/SA2_population.csv', index_col=0)
evo_cust_df = pd.read_csv('data/evo_customers.csv', index_col=0)

# http://www.bom.gov.au/climate/averages/tables/cw_070282.shtml
mean_3pm_temps = [26.9,26.4,23.5,19.1,14.9,11.4,10.6,12.6,15.1,18.3,22.0,24.8,18.8]
mean_temps_Bin = [20.495001,20.325001,17.85,13.48,9.469999,6.67,5.585,7.105,9.710001,12.985,15.895,18.775]

In [138]:
def zs_data(start: str, end: str, src: str) -> pd.DataFrame:
    """Get data from """
    data_df = pd.read_csv(src, index_col=0)#, parse_dates=["Time"])

    freq='15min'
    intervals_per_hour = 4

    # temp bandaid to deal with substation data
    # data_df.index = pd.to_datetime(data_df.index, utc=True)#, format='mixed')

    # NOTE timestamps indicate start of an interval
    # Starting with 15 min data we'll set the time index to start at the beninning of this 
    # period and then trim off the first 5 min after ffill()
    tmp = pd.date_range(start=pd.to_datetime("2021-7-1 00:00").tz_localize("UTC"), 
                        end=pd.to_datetime("2023-06-30 23:45").tz_localize("UTC"), 
                        freq=freq)
    data_df.index = tmp

    # data_df = data_df.resample("5min").ffill()#.reset_index()

    # data_df.rename(columns={data_df.columns[0]: "network_load"}, inplace=True)

    new_df = data_df.loc[start:end]#[1:]

    # new_df.reset_index(inplace=True)

    # temp bandaid to deal with substation data
    # new_df = new_df.rename(columns={data_df.index.name:'settlementdate'})
    # new_df = new_df.rename(columns={'index':'settlementdate'})
    return new_df, intervals_per_hour

In [186]:
def forecast(evo_region, nu_days, start_day, zs_df, timed_hw_start, timed_hw_end,
             current_households, future_households, mean_temps, #resist_kWh_per_household,
             current_frac_gas, future_frac_gas, current_frac_heatpumps, future_frac_heatpumps,
             intervals_per_hour):
    current_frac_elec = 1 - current_frac_gas
    future_frac_elec = 1 - future_frac_gas
    current_frac_resist = 1 - current_frac_heatpumps
    future_frac_resist = 1 - future_frac_heatpumps

    date_dict = {}
    for i_d, day in enumerate(range(start_day,nu_days)):
        current_day_zs_df = zs_df[evo_region][zs_df.index.dayofyear == day+1]
        current_day_zs_df.rename('Current', inplace=True)
        # current_day_zs_df.index = current_day_zs_df.index.strftime('%H:%M')
        # current_day_zs_df.index = current_day_zs_df.index.time
        # time_interval = current_day_zs_df.index[1] - current_day_zs_df.index[0]
        # print(current_day_zs_df)
        # print(repr(current_day_zs_df.index))

        date_dict[day] = current_day_zs_df.index[0].date()
        month = current_day_zs_df.index[0].month
        temp = mean_temps[month-1]

        current_gas_users = current_frac_gas*current_households
        future_elec_users = future_households*future_frac_elec
        
        pop_growth_factor = (future_households)/(current_households)
        
        # heating energy demand as function of temperature per household
        CBR_factor = 1.266
        hw_kWh_household_per_day = (10.295 - 0.2816*temp)*CBR_factor
        # Heat pump coefficient of performance as function of temperature
        heatpump_CoP = 2.9664 + 0.0703*temp

        current_hw_MWh_per_day = current_frac_elec*current_households \
            *(current_frac_resist*hw_kWh_household_per_day + current_frac_heatpumps*hw_kWh_household_per_day/heatpump_CoP)/1000
        current_hw_MW = current_hw_MWh_per_day/24
            
        curent_day_zs_sans_hw = current_day_zs_df - current_hw_MW
        curent_day_zs_sans_hw.rename('Current w/o hot water', inplace=True)

        future_zs = current_day_zs_df*pop_growth_factor
        future_zs_sans_hw = curent_day_zs_sans_hw*pop_growth_factor
        
        future_hw_resist_kWh = future_elec_users*future_frac_resist*hw_kWh_household_per_day
        future_hw_heatpump_kWh = future_elec_users*future_frac_resist*hw_kWh_household_per_day/heatpump_CoP
        future_hw_MWh_total = (future_hw_heatpump_kWh + future_hw_resist_kWh)/1000


        time_mask = (current_day_zs_df.index.hour >= timed_hw_start) & \
                (current_day_zs_df.index.hour < timed_hw_end)
        timed_hw_df = pd.Series(index=current_day_zs_df.index, data = 0)
        # timed_hw_df[timed_hw_df.index.hour > timed_hw_start & timed_hw_df.index.hour < timed_hw_end] == 1
        # print(timed_hw_df.between_time(timed_hw_start,timed_hw_end))
        timed_hw_df[time_mask] = 1
        future_hw_MW = future_hw_MWh_total / (np.sum(timed_hw_df)/intervals_per_hour)
        timed_hw_df = timed_hw_df*future_hw_MW
        # print(timed_hw_df)

        future_zs_with_hw = future_zs_sans_hw + timed_hw_df
        future_zs_with_hw.rename('Future', inplace=True)

        # print(future_zs_with_hw)

        plotting_df = pd.concat([current_day_zs_df, curent_day_zs_sans_hw, future_zs_with_hw],axis=1)

        current_day_zs_df.index = current_day_zs_df.index.time
        curent_day_zs_sans_hw.index = current_day_zs_df.index
        future_zs_with_hw.index = current_day_zs_df.index

        if i_d == 0:
            many_day_df_current = current_day_zs_df.rename(date_dict[day], inplace=True).to_frame()
            many_day_df_current_sans = curent_day_zs_sans_hw.rename(date_dict[day], inplace=True).to_frame()
            many_day_df_future = future_zs_with_hw.rename(date_dict[day], inplace=True).to_frame()
        else:
            many_day_df_current = many_day_df_current.join(current_day_zs_df.rename(date_dict[day]))
            many_day_df_current_sans = many_day_df_current_sans.join(curent_day_zs_sans_hw.rename(date_dict[day]))
            many_day_df_future = many_day_df_future.join(future_zs_with_hw.rename(date_dict[day]))

    plotting_df_many = pd.concat([many_day_df_current.mean(axis=1), many_day_df_current_sans.mean(axis=1), many_day_df_future.mean(axis=1)],axis=1)
    plotting_df_many.columns = ['Current','Current w/o hot water','Future']

    return many_day_df_current, many_day_df_current_sans, many_day_df_future, plotting_df_many, plotting_df

In [190]:


POE = 0.9
nu_days = 300
start_day = 0#5*30

timed_hw_start = 9
timed_hw_end = 17
# resist_kWh_per_household = 8.1
current_frac_gas = 0.45
future_frac_gas = 0.0
current_frac_heatpumps = 0.1
future_frac_heatpumps = 0.75

# pop_level = 3
# pop_region = 'Gungahlin'
# evo_region = 'Gold Creek'

pop_level = 2
pop_region = 'Total - Belconnen'
evo_region = 'Belconnen'

current_households = evo_cust_df.loc[evo_region,"Evo resi customers"]

if pop_level == 2:
    current_pop = pop_SA2_df.loc[pop_region,'2022']
    pop_per_household = current_pop/current_households
    # print(pop_per_household)
    future_households = pop_SA2_df.loc[pop_region,'2045']/pop_per_household
# elif pop_level == 3:
#     current_households = pop_SA3_df.loc[pop_region,'2022']/pop_per_household
#     future_households = pop_SA3_df.loc[pop_region,'2045']/pop_per_household



# mean_temps = mean_3pm_temps
mean_temps = mean_temps_Bin


many_day_df_current, many_day_df_current_sans, many_day_df_future, plotting_df_many, \
      plotting_df = forecast(evo_region, nu_days, start_day, zs_df, timed_hw_start, timed_hw_end,
             current_households, future_households, mean_temps, #resist_kWh_per_household,
             current_frac_gas, future_frac_gas, current_frac_heatpumps, future_frac_heatpumps,
             intervals_per_hour)


2.728738059067384


In [191]:

# future_zs_with_hw.plot()
# many_day_df_current.plot()
# many_day_df_current.mean(axis=1).plot()
# plotting_df_many.plot()
fig = go.Figure(plotting_df_many.plot())
fig.update_yaxes(range = [0,plotting_df_many.max().max()])


In [192]:
# Utilization
total_day_profile = many_day_df_current.sum(axis='columns')/1000
total_day_profile_peak = total_day_profile.max()
print('Current total energy delivered = ', np.round(total_day_profile.sum(),0), " GWh")
total_day_profile = many_day_df_future.sum(axis='columns')/1000
total_day_profile_peak = total_day_profile.max()
print('Future total energy delivered = ', np.round(total_day_profile.sum(),0), " GWh")

POE_index = int(POE*nu_days)
# peak_load_MW = many_day_df_future.max(axis='columns')
peak_load_day = many_day_df_future.max().idxmax()
peak_load_MW = many_day_df_future[peak_load_day].max()
sorted_load_days = many_day_df_future.max().sort_values(ascending=True)

high_load_days = sorted_load_days.iloc[POE_index:].index.to_numpy()
many_day_df_future[high_load_days].plot()

min_threshold_POE = many_day_df_future[high_load_days].max().min()

print(f"Threshold of {POE} POE = {np.round(min_threshold_POE,1)} MW")
print(f"Max peak = {np.round(peak_load_MW,1)} MW")


Current total energy delivered =  850.0  GWh
Future total energy delivered =  1063.0  GWh
Threshold of 0.9 POE = 75.7 MW
Max peak = 85.1 MW


In [136]:

# fig = make_subplots(rows=1, cols=1)#,specs=[[{"secondary_y": True}]])
fig = go.Figure(total_day_profile.plot())
fig.update_yaxes(range = [0,total_day_profile_peak])